In [1]:
from datetime import datetime, timedelta
import time
import os
import shutil
import json
import pandas as pd
import logging

In [2]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [3]:
#Write file to local directory
def write_file_local(path,file_data, write_mode='w'):
    
    with open(path, write_mode) as file:
        file_string = file_data
        file.write(file_string)

In [4]:
with open('lob_config.json') as json_lob_config:
    config = json.load(json_lob_config)

lob = config['lob']
project_id = config['project_id']
input_path_folder = config['input_path_folder']
output_path_folder = config['output_path_folder']
input_core_tables_file = config['input_core_tables_file']
merged_output_file = config['merged_output_file']
sql_extension = config['sql_extension']
datasets = config['datasets']

parms_find_replace_list = [
    {'search':'{{ params.param_pbs_core_dataset_name }}.','replace':'`hca-hin-curated-mirroring-td`.edwpbs.'},
    {'search':'{{ params.param_pbs_base_views_dataset_name }}.','replace':'`hca-hin-prod-cur-parallon`.edwpbs_base_views_copy.'},
    {'search':'{{ params.param_auth_base_views_dataset_name }}.','replace':'`hca-hin-prod-cur-parallon`.auth_base_views.'},
    {'search':'{{ params.param_pbs_bqutil_fns_dataset_name }}.','replace':'`hca-hin-prod-cur-parallon`.bqutil_fns.'}
]

In [5]:
def execute_sql_files(sql_file_path):

    client = bigquery.Client(project=project_id)
    with open(sql_file_path, 'r') as sql_file:
        file_name = os.path.basename(sql_file_path)
        sql_query = sql_file.read()
        try:         
            client.query(sql_query, project=project_id, location='US').result()
            logging.info(f"SQL file {file_name} executed successfully.")
            print(f"SQL file {file_name} executed successfully.")
        except Exception as e:
            logging.error(f"Error executing SQL file {file_name}: {e}")
            print(f"Error executing SQL file {file_name}: {e}")

In [6]:
#get the DDLs for the tables from teradata 
def check_views_for_tables():
    dt1 = datetime.now()

    try:
        
        for dataset in datasets:
            qualified_path = os.path.join(output_path_folder, lob.strip(), dataset)
            print(qualified_path)
            if os.path.exists(qualified_path):
                shutil.rmtree(qualified_path, ignore_errors=True)
            os.makedirs(qualified_path)

        df = pd.read_csv(input_core_tables_file, index_col=None)

        merged_output_file_path = f"{merged_output_file}{sql_extension}"
        if os.path.exists(merged_output_file_path):
            os.remove(merged_output_file_path)

        for index, row in df.iterrows():
            table_name = str(row['table_id']).strip().lower()
            file_name_search = f"{table_name}{sql_extension}"
            for dataset in datasets:
                input_file_path = os.path.join(input_path_folder, dataset, file_name_search)
                if os.path.exists(input_file_path):
                    file_contents = read_contents_file(input_file_path)
                    output_file_path = f"{output_path_folder}\{lob.strip()}\{dataset}\{file_name_search}"
                    for i in range(len(parms_find_replace_list)):
                        file_contents = file_contents.replace(parms_find_replace_list[i]["search"], parms_find_replace_list[i]['replace'])  
                    write_file_local(output_file_path, file_contents)
                    write_file_local(merged_output_file_path, file_contents, "a")

        # print("Project ID : {}".format(project_id))
        
        # execute_sql_files(qualified_filename_path)

    except Exception as e1:
        print(e1)
        print("File Name : {}".format(file_name_search))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

In [7]:
print("Begin of Processing")

check_views_for_tables()

print("End of Processing")

Begin of Processing
Qualified_SQLS\PBS\edwpbs_base_views
0:00:08.094898
End of Processing
